In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, return_messages=True
)

# Few-shot 예제 데이터 (학습용)
examples = [
    {"question": "What do you know about The Godfather?", "answer": "👨‍👨‍👦🔫🍝"},
    {"question": "What do you know about TopGun?", "answer": "🛩️👨‍✈️🔥"},
]

# AI야 너 이렇게 질문이 들어오면 저렇게 답변 했었어! 라고 주입 교육
# example_prompt = ChatPromptTemplate.from_messages(
#     [("human", "What do you know about {movie}?"), ("ai", "{answer}")]
# )

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{question}"), ("ai", "{answer}")]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You must answer using only three emojis. "
            "However, for the first movie question in a conversation, you must answer with the movie title instead of emojis."
            "The movies from the examples (The Godfather, Top Gun) must not be used in any answers."
            "Always refer to the conversation history to determine if this is the first movie question.",
        ),
        example_prompt,
        ("system", "Conversation history:\n{history}"),  # 대화 이력을 시스템에 포함
        # MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({}).get("history", "")
    # return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    question = f"What do you know about {movie}?"
    result = chain.invoke({"question": question})  # movie -> question 변경
    memory.save_context({"input": question}, {"output": result.content})
    # print(result.content)


invoke_chain("Avatar")

🌿🔵🪐

In [20]:
invoke_chain("Sonic")

🦔💨🎮

In [21]:
invoke_chain("What was the first movie I asked about?")  # 예상: "Avatar"

The Godfather